In [1]:
import pandas as pd
import trino

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', None)

print('Các thư viện đã sẵn sàng.')
conn = trino.dbapi.connect(host='localhost', port=8080, user='python-client', catalog='lakehouse', schema='rva')
cursor = conn.cursor()
print('Đã kết nối Trino: catalog=lakehouse, schema=rva')


Các thư viện đã sẵn sàng.
Đã kết nối Trino: catalog=lakehouse, schema=rva


In [2]:
# Liệt kê bảng trong schema rva
cursor.execute("SHOW TABLES FROM lakehouse.rva")
tables = cursor.fetchall()
print('Các bảng hiện có:')
print(tables)


Các bảng hiện có:
[['bronze_raw'], ['gold_hour_by_cam'], ['gold_minute_by_cam'], ['silver_detections']]


In [3]:
# Mô tả schema hai bảng Gold nếu tồn tại
def describe(table):
    try:
        return pd.read_sql(f"DESCRIBE lakehouse.rva.{table}", conn)
    except Exception as e:
        print(f'Không mô tả được {table}:', e)
        return None

df_min = describe('gold_minute_by_cam')
df_hr = describe('gold_hour_by_cam')
df_min, df_hr


/tmp/ipykernel_434916/2609757020.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(f"DESCRIBE lakehouse.rva.{table}", conn)


(                Column          Type Extra Comment
 0             store_id       varchar              
 1            camera_id       varchar              
 2            ts_minute  timestamp(6)              
 3      detection_count        bigint              
 4        unique_tracks        bigint              
 5             avg_conf        double              
 6  bbox_area_ratio_avg        double              ,
             Column          Type Extra Comment
 0         store_id       varchar              
 1        camera_id       varchar              
 2          ts_hour  timestamp(6)              
 3  detection_count        bigint              
 4    unique_tracks        bigint              
 5         avg_conf        double              )

In [4]:
# Thống kê nhanh số dòng và vài mẫu gần nhất
def q(sql):
    try:
        return pd.read_sql(sql, conn)
    except Exception as e:
        print('Lỗi SQL:', e)
        return None

df_cnt_min = q("SELECT COUNT(*) AS rows FROM lakehouse.rva.gold_minute_by_cam")
df_cnt_hr = q("SELECT COUNT(*) AS rows FROM lakehouse.rva.gold_hour_by_cam")
print('Số dòng minute/hour:')
print(df_cnt_min, df_cnt_hr)

df_last_min = q("SELECT * FROM lakehouse.rva.gold_minute_by_cam ORDER BY ts_minute DESC, store_id, camera_id LIMIT 20")
df_last_hr = q("SELECT * FROM lakehouse.rva.gold_hour_by_cam ORDER BY ts_hour DESC, store_id, camera_id LIMIT 20")
df_last_min, df_last_hr


/tmp/ipykernel_434916/3570886649.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


Số dòng minute/hour:
   rows
0     2    rows
0     1


/tmp/ipykernel_434916/3570886649.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


(   store_id camera_id           ts_minute  detection_count  unique_tracks  \
 0  store_01    cam_01 2025-11-13 06:26:00              497              5   
 1  store_01    cam_01 2025-11-13 06:25:00               89              2   
 
    avg_conf  bbox_area_ratio_avg  
 0  0.797487             0.048347  
 1  0.831104             0.038859  ,
    store_id camera_id             ts_hour  detection_count  unique_tracks  \
 0  store_01    cam_01 2025-11-13 06:00:00              586              5   
 
    avg_conf  
 0  0.802593  )

In [5]:
# Kiểm tra partition Iceberg của Gold (nếu connector hỗ trợ)
part_min = q("SELECT * FROM lakehouse.rva."gold_minute_by_cam$partitions" ORDER BY record_count DESC LIMIT 10")
part_hr  = q("SELECT * FROM lakehouse.rva."gold_hour_by_cam$partitions" ORDER BY record_count DESC LIMIT 10")
part_min, part_hr


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1763112945.py, line 2)

In [6]:
# So khớp Gold vs Silver trong ngày hiện tại (consistency)
sql_min = """
WITH g AS (
  SELECT store_id, camera_id, ts_minute, detection_count
  FROM lakehouse.rva.gold_minute_by_cam
  WHERE ts_minute >= date_trunc('day', current_timestamp)
), s AS (
  SELECT store_id, camera_id, date_trunc('minute', capture_ts) AS ts_minute, COUNT(*) AS detection_count
  FROM lakehouse.rva.silver_detections
  WHERE capture_ts >= date_trunc('day', current_timestamp)
  GROUP BY 1,2,3
)
SELECT
  COALESCE(g.store_id,s.store_id) store_id,
  COALESCE(g.camera_id,s.camera_id) camera_id,
  COALESCE(g.ts_minute,s.ts_minute) ts_minute,
  COALESCE(g.detection_count,0) g_cnt,
  COALESCE(s.detection_count,0) s_cnt,
  (COALESCE(g.detection_count,0)-COALESCE(s.detection_count,0)) AS diff
FROM g FULL OUTER JOIN s
ON g.store_id=s.store_id AND g.camera_id=s.camera_id AND g.ts_minute=s.ts_minute
WHERE COALESCE(g.detection_count,0) <> COALESCE(s.detection_count,0)
ORDER BY ts_minute DESC, store_id, camera_id
"""
df_diff_min = q(sql_min)
df_diff_min


/tmp/ipykernel_434916/3570886649.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


,store_id,camera_id,ts_minute,g_cnt,s_cnt,diff
